In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

ModuleNotFoundError: No module named 'mlflow'

# Data

In [23]:
df = pd.read_csv('./../../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
# fix ags5 missing 0 
df['ags5'] = np.where(df.ags5.str.len() == 4, '0' + df['ags5'], df['ags5'])
df.set_index('ags5', drop=True, inplace=True)
#df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2007-05-01')]
df

,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,number_of_companies_domestic_staff,...,employees_social_security_at_residence,employees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate
ags5,,,,,,,,,,,,,,,,,,,,,
01001,2007-05-01,4.0,2.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
01001,2007-06-01,8.0,6.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.2
01001,2007-07-01,9.0,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5
01001,2007-08-01,6.0,4.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.3
01001,2007-09-01,11.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16073,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
16074,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6
16075,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8


In [24]:
# select only column for which all values are avaliable
df = df[['date', 
        'number_of_company_deletions', 
        'number_of_company_liquidations', 
        'number_of_start_ups',
        'unemployment_rate', 
        'registerd_jobs',
        'employees_social_security_at_work',
        'employees_social_security_at_residence']]

In [25]:
# rename date and unemployment_rate for Prophet function 
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [35]:
# one-step forecast
def Prophet_forecast(data, config):
    g, sps, ys, sm, nc = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm, 
                   n_changepoints= nc)
    # fit model
    model_fit = model.fit(data)
    # make one step forecast
    future = model_fit.make_future_dataframe(periods=29, freq='MS')
    res = model_fit.predict(future)
    res.to_csv(f"{g}_{sps}_{ys}_{sm}")
    return res.iloc[-29:][["ds","yhat"]]

In [29]:
df_ags5 = df.filter(regex='01001', axis=0)
config = ['linear', 15.0, True, 'additive', 80]
Prophet_forecast(df_ags5, config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
169,2021-06-01,8.760998
170,2021-07-01,9.071886
171,2021-08-01,9.113309
172,2021-09-01,8.708875
173,2021-10-01,8.542492
174,2021-11-01,8.532300
175,2021-12-01,8.637941
176,2022-01-01,9.438736
177,2022-02-01,9.475346
178,2022-03-01,9.410599


In [30]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [31]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [32]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [36]:
def run_model(data, n_test): 
        for g in ['linear','logistic']: 
            for sps in [10.0]: 
                for ys in [True]: 
                    for sm in ['additive','multiplicative']:
                        for nc in [50, 85]:
                            with mlflow.start_run():
                                params = [g, sps, ys, sm, nc]

                                ags5 = list(set(df.index.values.tolist()))
                                rmse = []
                                for i in ags5:
                                    df_ags5 = df.filter(regex=i, axis=0)
                                    current_rmse = walk_forward_validation(df_ags5, n_test, params)
                                    print(current_rmse)
                                    rmse.append(current_rmse)

                                # Log params
                                params = {"growth":g, "sps":sps, "ys":ys, "sm":sm, "nc":nc}

                                mlflow.log_params(params)

                                # Log metrics
                                mlflow.log_metric('rmse', np.mean(rmse))
                                mlflow.log_metric('mse', np.mean(rmse)**2)

In [37]:
run_model(df, 29)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.6169361105367002


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.7139778679674276


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.7841281865224498


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1.1222637855771387
0.8504674575190375


ValueError: Capacities must be supplied for logistic growth in column "cap"

In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [2]:
Prophet?